In [200]:
#Caricare le librerie
import tweepy
import pandas as pd
from tqdm import tqdm
import re

In [201]:
consumer_key= "R8zEjULDipNcbMMeGNvnZcbp6"
consumer_secret= "6V856b39nGvS5bgKj9Mf15hc6FEiFnhSlaC8A7G81ZIS7bPxha"
access_token= "1054459819145510914-zRzbWMLQui10gwAL0Rh4IRAbPsZXLT"
access_token_secret= "kZj5myoG661SRnobImDChIIer1b6jHdaDmjqvygKds03T"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [202]:
#Scaricare in un dataframe i tweet in base ad un hashtag (until="2021-12-21")

#AAA: aggiungere l'hashtag direttamente tra le colonne delle liste Tweets e Retweetted


hashtags=["#NoGreenPass","#vaccinatevi","#casapound","@matteosalvinimi","@EnricoLetta"]
list_tweet=[]
list_originals=[]
list_rt_id=[]
list_retweetted=[]


for hashtag in hashtags:
  next_cursor=None
  n=5000
  pbar = tqdm(total=n)
  while n>0:
    for tweet in tweepy.Cursor(api.search_tweets,q=hashtag,lang="it", tweet_mode='extended').items(n): #,cursor=next_cursor

      #gestione dei retweet con try-except
      try:
        list_originals.append(tweet.retweeted_status.id) #creo lista dei retweets id 
        list_rt_id.append(tweet.retweeted_status.user.id)

        #lista tweets che sono stati retweettati
        try: 
          rt_text=tweet.retweeted_status.full_text
        except AttributeError:
          rt_text=tweet.retweeted_status.text

        list_retweetted.append([tweet.retweeted_status.created_at, tweet.retweeted_status.id,tweet.retweeted_status.user.id, rt_text, 
                         tweet.retweeted_status.favorite_count, tweet.retweeted_status.retweet_count, 
                         tweet.retweeted_status.user.screen_name,hashtag]) 
      except:
        list_originals.append(0)
        list_rt_id.append(0) 

      try: 
        tweet_text=tweet.full_text
      except AttributeError:
        tweet_text=tweet.text

      list_tweet.append([tweet.created_at, tweet.id,tweet.user.id, tweet_text, 
                         tweet.favorite_count, tweet.retweet_count, 
                         tweet.user.location,tweet.user.screen_name,
                         hasattr(tweet, "retweeted_status"),
                         tweet.in_reply_to_screen_name,hashtag])
      n -=1
      #next_cursor=tweet
      pbar.update(n=1)

100%|██████████| 5000/5000 [14:13:46<00:00, 10.25s/it]
Rate limit reached. Sleeping for: 806
100%|██████████| 5000/5000 [29:49<00:00,  2.79it/s]  
Rate limit reached. Sleeping for: 832
Rate limit reached. Sleeping for: 826
100%|██████████| 5000/5000 [30:08<00:00,  2.76it/s]  
Rate limit reached. Sleeping for: 835
Rate limit reached. Sleeping for: 813


In [205]:
#creo df dei tweet
tweets = pd.DataFrame(list_tweet, 
                      columns=['date','id','user_id','text','like','n_rt','location','author',
                               "retweet","reply_name",'hashtags'])


tweets["id_retweet"]=list_originals
tweets["user_id_retweet"]=list_rt_id


#creo df dei retweet
retweetted = pd.DataFrame(list_retweetted, 
                      columns=['date','id','author_id','text','like','n_rt','author','hashtags'])    


tweets.head()

,date,id,user_id,text,like,n_rt,location,author,retweet,reply_name,hashtags,id_retweet,user_id_retweet
0,2021-12-30 10:55:21+00:00,1476507191133843457,442083142,RT @Lorenzo62752880: La logica alla base del G...,0,122,Milano,mspini2,True,None,#NoGreenPass,1476217276525387776,1364298062919794688
1,2021-12-30 10:53:56+00:00,1476506837730140161,305255857,RT @GianniMagini: Questo scemo urlava col mega...,0,7,,Rosyfree74,True,None,#NoGreenPass,1476483050703929346,977383349235961857
2,2021-12-30 10:53:41+00:00,1476506773590880258,950098070771036160,"RT @GandalfRevn: ""La prima volta nella storia ...",0,354,,GrandeTornado1,True,None,#NoGreenPass,1444582094265634816,1438546161955229703
3,2021-12-30 10:52:07+00:00,1476506377614942208,229160327,RT @Lorenzo62752880: Crisanti al Corriere:\n\n...,0,149,altrove🇮🇹🇮🇹🇮🇹🇮🇹🇮🇹,marcoranieri72,True,None,#NoGreenPass,1476289519402078209,1364298062919794688
4,2021-12-30 10:51:48+00:00,1476506297356926982,229160327,RT @Lorenzo62752880: @borghi_claudio Ovviament...,0,68,altrove🇮🇹🇮🇹🇮🇹🇮🇹🇮🇹,marcoranieri72,True,None,#NoGreenPass,1476297527129432073,1364298062919794688


In [207]:
tweets_2=tweets.merge(retweetted[['id','text']],left_on='id_retweet',right_on='id',how='left')[['date','id_x','user_id','text_x','author','retweet','id_retweet','hashtags','user_id_retweet','text_y']].rename(columns={'text_y':'text_complete'})

In [208]:
tweets_2=tweets_2[tweets_2.text_complete.notnull()].drop_duplicates().reset_index(drop=True)

In [212]:
#arricchimento df tweets
weigth_column=[]

for index,row in tweets_2.iterrows():
    if row['hashtags']=="#NoGreenPass":
        weigth_column.append('#fb1239')
    elif row['hashtags']=="#vaccinatevi":
        weigth_column.append('#007474')
    elif row['hashtags']=="#casapound":
        weigth_column.append('#042069')
    elif row['hashtags']=="@matteosalvinimi":
        weigth_column.append('#99cccc')
    elif row['hashtags']=="@EnricoLetta":
        weigth_column.append('#b00b69')
    else:
        weigth_column.append('#aaaacc')

tweets_2['colours']=weigth_column

In [214]:
weigth_column_r=[]

for index,row in retweetted.iterrows():
    if row['hashtags']=="#NoGreenPass":
        weigth_column_r.append('#fb1239')
    elif row['hashtags']=="#vaccinatevi":
        weigth_column_r.append('#007474')
    elif row['hashtags']=="#casapound":
        weigth_column_r.append('#042069')
    elif row['hashtags']=="@matteosalvinimi":
        weigth_column_r.append('#99cccc')
    elif row['hashtags']=="@EnricoLetta":
        weigth_column_r.append('#b00b69')
    else:
        weigth_column_r.append('#aaaacc')

retweetted['colours']=weigth_column_r


In [215]:
tweets_2

,date,id_x,user_id,text_x,author,retweet,id_retweet,hashtags,user_id_retweet,text_complete,colours
0,2021-12-30 10:55:21+00:00,1476507191133843457,442083142,RT @Lorenzo62752880: La logica alla base del G...,mspini2,True,1476217276525387776,#NoGreenPass,1364298062919794688,La logica alla base del Green Pass: più non fu...,#fb1239
1,2021-12-30 10:53:56+00:00,1476506837730140161,305255857,RT @GianniMagini: Questo scemo urlava col mega...,Rosyfree74,True,1476483050703929346,#NoGreenPass,977383349235961857,Questo scemo urlava col megafono nelle piazze ...,#fb1239
2,2021-12-30 10:53:41+00:00,1476506773590880258,950098070771036160,"RT @GandalfRevn: ""La prima volta nella storia ...",GrandeTornado1,True,1444582094265634816,#NoGreenPass,1438546161955229703,"""La prima volta nella storia che l'inefficacia...",#fb1239
3,2021-12-30 10:52:07+00:00,1476506377614942208,229160327,RT @Lorenzo62752880: Crisanti al Corriere:\n\n...,marcoranieri72,True,1476289519402078209,#NoGreenPass,1364298062919794688,"Crisanti al Corriere:\n\n""In questo momento no...",#fb1239
4,2021-12-30 10:51:48+00:00,1476506297356926982,229160327,RT @Lorenzo62752880: @borghi_claudio Ovviament...,marcoranieri72,True,1476297527129432073,#NoGreenPass,1364298062919794688,@borghi_claudio Ovviamente un obbligo vaccinal...,#fb1239
...,...,...,...,...,...,...,...,...,...,...,...
9268,2021-12-30 12:50:55+00:00,1476536274622328833,1247063129693065216,RT @ThomasE26847666: Ma fatemi capire chi e' p...,Simona62506151,True,1476450711521140739,@EnricoLetta,1468626755833720838,Ma fatemi capire chi e' positivo e ha il covid...,#b00b69
9269,2021-12-30 12:48:58+00:00,1476535784983379972,1020303953974759424,RT @PeterWo31671308: @ALFO100897 @EnricoLetta ...,ALFO100897,True,1476535628531707908,@EnricoLetta,1388915991686664196,@ALFO100897 @EnricoLetta Alfo ma il Super Gree...,#b00b69
9270,2021-12-30 12:48:58+00:00,1476535784517869571,2288712929,RT @theStewieee: @EnricoLetta @il_cappellini S...,apavo75,True,1476534844435972101,@EnricoLetta,843385951929950208,@EnricoLetta @il_cappellini Su amazon le trovi...,#b00b69
9271,2021-12-30 12:48:34+00:00,1476535685171527683,1367203155440771082,RT @ALFO100897: @marilenagasbar1 @EnricoLetta ...,marilenagasbar1,True,1476535452182138881,@EnricoLetta,1020303953974759424,"@marilenagasbar1 @EnricoLetta Solo ""qualcosa"",...",#b00b69


In [210]:
retweetted=retweetted.drop_duplicates(['id','author_id']).reset_index(drop=True)

In [211]:
#selezione dei tweet più retwettati 
retweetted.sort_values("id",ascending=False)

,date,id,author_id,text,like,n_rt,author,hashtags
1337,2021-12-30 12:48:21+00:00,1476535628531707908,1388915991686664196,@ALFO100897 @EnricoLetta Alfo ma il Super Gree...,2,2,PeterWo31671308,@EnricoLetta
1340,2021-12-30 12:47:39+00:00,1476535452182138881,1020303953974759424,"@marilenagasbar1 @EnricoLetta Solo ""qualcosa"",...",1,1,ALFO100897,@EnricoLetta
1339,2021-12-30 12:45:14+00:00,1476534844435972101,843385951929950208,@EnricoLetta @il_cappellini Su amazon le trovi...,2,1,theStewieee,@EnricoLetta
1341,2021-12-30 12:32:32+00:00,1476531648388907008,513144334,"@EnricoLetta @il_cappellini Quindi, Enrico, il...",3,1,RenatoSouvarine,@EnricoLetta
977,2021-12-30 12:25:46+00:00,1476529946877177859,938345883279724544,@MauroV1968 @FranceskoNew @ASampierdarena @Car...,3,2,ScuolaNoCovid,@EnricoLetta
...,...,...,...,...,...,...,...,...
701,2020-11-01 10:22:08+00:00,1322846377475481601,270839361,"🔴‼️Stamane a #Lampedusa: oltre 100, e si parla...",2996,863,matteosalvinimi,@matteosalvinimi
1163,2019-09-01 12:12:01+00:00,1168134413621846017,419622371,Un gran segno di #speranza! Che bella scelta! ...,1021,204,EnricoLetta,@EnricoLetta
928,2019-08-13 12:34:25+00:00,1161254678409175041,270839361,Simpatico questo vignettista di molti giornali...,4598,976,matteosalvinimi,@matteosalvinimi
668,2019-05-31 18:35:06+00:00,1134528748521480194,831646859378774016,Se #Assange muore saranno responsabili oltre a...,584,275,Wikileaks_Ita,@matteosalvinimi


##### Vado a vedere quali sono i tweet più retwittati e quante volte sono stati ritwittati dalla nostra rete (usando SQL)

In [136]:
#Open the terminal and run-   pip install U pandasql
#Open your Jupyter Notebook and in any cell run-    !pip install -U pandasql
import pandasql as ps
q1 = """SELECT id,like,n_rt, author,count() FROM retweetted group by id order by n_rt desc"""
print(ps.sqldf(q1, locals()))

InterfaceError: (sqlite3.InterfaceError) Error binding parameter 8 - probably unsupported type.
[SQL: INSERT INTO retweetted (date, id, author_id, text, "like", n_rt, author, colours, hashtags) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)]
[parameters: (('2021-12-29 03:14:47.000000', 1476028900312551426, 2963478049, 'Ho appena visto #DontLookUp incredibile che lo abbiano permesso:il personaggio filantropo ricorda B. Gates/Biden e i nuovi ricchi che spacciano le lo ... (5 characters truncated) ... ee per scienza.Da vedere @NetflixIT SUBITO. Rispecchia lo stato attuale!\n#vaccino #NoGreenPass #draghibugiardo @barbarab1974 https://t.co/YRTG5YHFPW', 1, 1, 'singerindubai', '#aaaacc', []), ('2021-12-28 20:27:02.000000', 1475926285365006345, 1364298062919794688, 'Anche il sindaco di Verona positivo nonostante vaccino. \n\nLe discriminazioni e le sospensioni dal lavoro sono sempre più senza fondamento. Il green pass misura da rimuovere. \n\n#NoGreenPass', 369, 106, 'Lorenzo62752880', '#aaaacc', []), ('2

#### Creazione input Grephi tramite scrittura file .csv
Da rivedere:
- Abbiamo creato nodi con l'id del tweet, non per utente
- Si possono pesare gli archi tra utenti per il numero di retweet tra essi? Ha senso?

In [216]:
rt_tweets=tweets_2[tweets_2.retweet==True][['user_id','user_id_retweet','author','colours','hashtags']]

In [217]:

tweets_ids=rt_tweets[['user_id','author','colours','hashtags']]
retwitted_ids=retweetted[['author_id','author','colours','hashtags']].rename(columns={'author_id':'user_id'})
rt_tweets_ids=pd.concat([tweets_ids,retwitted_ids]).sort_values(by=['hashtags'],ascending=False).groupby(['user_id','author']).head(1).reset_index(drop=True)

In [218]:
Nodes_df=pd.DataFrame({'Id':rt_tweets_ids.user_id,'Label':rt_tweets_ids.author,'Interval':None,'Colour':rt_tweets_ids.colours})

In [219]:
Edges_df=pd.DataFrame({'Source':rt_tweets.user_id,'Target':rt_tweets.user_id_retweet,'Type':'Directed','Kind':None,'Id':range(1, len(rt_tweets)+1),'Label':None,'Interval':None,'Weight':1.0})

In [220]:
Nodes_df.to_csv('Nodes_df6.csv', index=False)
Edges_df.to_csv('Edges_df6.csv', index=False)

##### Grafi con NetworkX

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
Edges_df=pd.read_csv("C:/Users/dvd_m/Desktop/Social Media Analytics/SMA-Project/Edges_df.csv")
G = nx.from_pandas_edgelist(Edges_df, source='Source', target='Target')
df_cluster=pd.DataFrame({"nodes":nx.clustering(G).keys(),"cluster":nx.clustering(G).values()})
grafo=pd.merge(Edges_df,df_cluster, left_on="Source",right_on="nodes")